Inisiasi Program

In [1]:
from scripts.pf_simulation import PowerFactorySim as pfsim
import os
import pandas as pd

FOLDER_NAME = ''
PROJECT_NAME = '39 Bus New England System'
STUDY_CASE_NAME = '1. Power Flow'
VARIABLES = {
    'name': {
        '*.ElmTerm': ['magnitude','angle'],
        '*.ElmSym': ['rotor_angle', 'out_of_step']
    },
    'symbol': {
        '*.ElmTerm': ['m:u1', 'm:phiu'],
        '*.ElmSym': ['c:firel', 's:outofstep']
    }
}

sim = pfsim(folder_name=FOLDER_NAME, project_name=PROJECT_NAME, study_case_name=STUDY_CASE_NAME)
# sim.app.Show()

# Membuat skenario operasi
SCENARIO = {
    'f_line' : sim.app.GetCalcRelevantObjects('*.ElmLne'),
    'f_location' : [0, 20, 50, 80],
    'f_duration' : [0.1, 0.15, 0.18, 0.2, 0.25],
    'load_level' : [i/100 for i in range(50,115,5)],
}
scenario_metadata = sim.createScenario(scenario_dict=SCENARIO, save=True, flag='y')

# Memuat nilai default load dan gen
file_dir = "output/default_data"
load_file = f"{file_dir}/load_data.parquet"
gen_file = f"{file_dir}/generator_data.parquet"
if all(map(os.path.exists, [load_file, gen_file])):
    load_default, gen_default = (pd.read_parquet(load_file), pd.read_parquet(gen_file))
else:
    load_default, gen_default = sim.loadGenData()
    load_default.to_parquet(load_file, index=False)
    gen_default.to_parquet(gen_file, index=False)

# Mengatur parameter untuk simulasi optimal power flow
sim.opfSetup()

# Mengatur parameter beban ke nilai default
sim.loadSetup(1, load_default)
sim.opfAnalysis('los')
sim.ldfAnalysis()

display(scenario_metadata)

Scenario metadata created


,scenario,load_level,f_line,f_location,f_duration
0,scenario_0001,0.5,Line 01 - 39.ElmLne,0,0.10
1,scenario_0002,0.5,Line 01 - 39.ElmLne,0,0.15
2,scenario_0003,0.5,Line 01 - 39.ElmLne,0,0.18
3,scenario_0004,0.5,Line 01 - 39.ElmLne,0,0.20
4,scenario_0005,0.5,Line 01 - 39.ElmLne,0,0.25
...,...,...,...,...,...
8835,scenario_8836,1.1,Line 01 - 02.ElmLne,80,0.10
8836,scenario_8837,1.1,Line 01 - 02.ElmLne,80,0.15
8837,scenario_8838,1.1,Line 01 - 02.ElmLne,80,0.18
8838,scenario_8839,1.1,Line 01 - 02.ElmLne,80,0.20


Testing OPF

In [5]:
file_dir = "output/generator_dispatch"
for load_lvl_iter in sim.load_level:
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        print(f'{load_lvl_iter:.2f} load, OPF min los failed, trying min dev')
        sim.opfAnalysis('dev')
    
    # Lakukan analisis daya dan ambil dispatch
    gen_dispatch = sim.ldfAnalysis().gen

    # Simpan hasil ke HDF5
    gen_dispatch.to_parquet(f'{file_dir}/scenario_{int(load_lvl_iter*100)}.parquet', index=False)
    print(f'{load_lvl_iter:.2f} load, OPF and LF running successfully')

0.50 load, OPF and LF running successfully
0.55 load, OPF and LF running successfully
0.60 load, OPF and LF running successfully
0.65 load, OPF and LF running successfully
0.70 load, OPF and LF running successfully
0.75 load, OPF min los failed, trying min dev
0.75 load, OPF and LF running successfully
0.80 load, OPF and LF running successfully
0.85 load, OPF and LF running successfully
0.90 load, OPF and LF running successfully
0.95 load, OPF and LF running successfully
1.00 load, OPF and LF running successfully
1.05 load, OPF and LF running successfully
1.10 load, OPF and LF running successfully


In [ ]:
from tqdm import tqdm
import time
import pandas as pd

restart = False
dataset_dir = "D:\Fathin\github\powerfactory_rms\output\dataset" # Harus path lengkap

# Baca file metadata
scenario_list = pd.read_csv("output/scenario_metadata.csv")

# Pastikan kolom 'iter_time' ada
if 'iter_time' not in scenario_list.columns:
    scenario_list['iter_time'] = None

# Filter skenario yang belum dijalankan
if restart:
    scenario_list['iter_time'] = None  # Reset semua iterasi
scenario_list_run = scenario_list[scenario_list['iter_time'].isna()].reset_index(drop=True)

# Total iterasi yang tersisa
total_iterations = scenario_list_run.shape[0]

# Mulai simulasi
with tqdm(total=total_iterations, desc="Simulating", unit="iter") as pbar:
    for _, scen in scenario_list_run.iterrows():
        # Dapatkan parameter scenario
        filename = scen['scenario']
        load_lvl_iter = scen['load_level']
        f_line_iter = scen['f_line']
        f_loc_iter = scen['f_location']
        f_dur_iter = scen['f_duration']

        iter_start_time = time.perf_counter()

        # Simulasi (implementasikan di sini)
        sim.loadSetup(float(load_lvl_iter), load_default)
        try:
            sim.opfAnalysis('los')
        except AttributeError:
            sim.opfAnalysis('dev')
        sim.scSetup(float(f_loc_iter), sim.app.GetCalcRelevantObjects(f_line_iter)[0], float(f_dur_iter))
        sim.rmsSimulation(VARIABLES['symbol'], t_start=-100, t_step=10, t_stop=30)
        sim.getResultRms(dataset_dir, filename)


        iter_end_time = time.perf_counter()
        iter_time = iter_end_time - iter_start_time

        # Update DataFrame dan simpan ke file
        scenario_list.loc[scenario_list['scenario'] == filename, 'iter_time'] = iter_time
        scenario_list.to_csv("output/scenario_metadata.csv", index=False)

        # Update progress bar
        pbar.set_postfix({"current_scenario": filename})
        pbar.update(1)


Simulating: 100%|██████████| 8840/8840 [13:34:23<00:00,  5.53s/iter, current_scenario=scenario_8840]  


In [1]:
import pandas as pd
import os
import glob

# Directory containing Parquet files
parquet_dir = "output/dataset"

# Get all Parquet file paths in the directory
parquet_files = glob.glob(os.path.join(parquet_dir, "*.parquet"))

# List to store DataFrames
dfs = []

# Loop through each file
for file in parquet_files:
    # Read the Parquet file into a DataFrame
    df = pd.read_parquet(file)
    
    # Add a new column with the filename (without extension)
    df['scenario'] = os.path.basename(file)[:-8]
    
    # Append to the list of DataFrames
    dfs.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame back to a single Parquet file
merged_df.to_parquet("output/dataset_IEEE39bus.parquet", index=False)

print("Merged file created with an added 'scenario' column!")


Merged file created with an added 'filename' column!


Data labelling

In [2]:
# Untuk query dan load data tiap skenario, tidak disaranakan load semua skenario secara langsung
from os import path as osp
from scripts.voltage_stability import VoltageStability
import pandas as pd
from tqdm import tqdm

# Directories
base_path = 'output' #'/kaggle/input/time-domain-ieee-39-bus'
dataset_path = osp.join(base_path, 'dataset_IEEE39bus.parquet')
metadata_path = osp.join(base_path, 'scenario_metadata.csv')

# Read scenario list
scenario_list = pd.read_csv(metadata_path)
scenario_list = scenario_list['scenario'].to_list()

# Do labelling
vs = VoltageStability(dataset_path)
# scenario = 'scenario_8800' # unstable
target_df = pd.DataFrame({'scenario': scenario_list})
targets = []
with tqdm(total=len(scenario_list), desc="Simulating", unit="iter") as pbar:
    for scenario in scenario_list:
        vs.get_dataset(scenario)
        target = vs.get_target_generator()
        targets.append(target)

        pbar.set_postfix({"current_scenario": scenario})
        pbar.update(1)

    target_df = pd.concat([target_df, pd.DataFrame(targets)], axis=1)




Simulating: 100%|██████████| 8840/8840 [46:58<00:00,  3.14iter/s, current_scenario=scenario_8840]  


In [12]:
os.path.basename(file)[:-8]

'scenario_8840'